# Web Mining: principes de bases

Le webmining est donc une technique qui repose sur les technologies du web et qui rentre dans la catégorie de  l'extraction et analyse de données (Text and Data Mining).

Le [cadre légal](Atelier_10.md) de cette pratique est assez complexe 

Les principales opérations consistent à:

-   se **connecter** à une page web et charger le html (load, connect) dans le cas de page statique c'est une opération  simple, dans le cas de page dynamique c'est une autre paire de manches...
-   **parcourir la page** et **extraire** les informations cibles   (celles qui nous intéressent: ce qui implique d'avoir identifiés au préalalble les informations qui nous intéresse dans la page soit les balises )
- les **stocker** dans un fichier ou une base de données en les organisant selon ce qui nous arrange pour l'analyse

L'objectif final étant de récupérer des informations pour les **analyser**. Ici [plein de techniques](./WebAnalysis.ipynb) s'appliquent en fonction de l'analyse qu'on veut en faire.

Nous allons d'abord installer dans le système, les modules complémentaires dont nous avons besoin. 
Dans le terminal

In [7]:
# !/usr/bin/bash

! pip install requests lxml BeautifulSoup4

## Télécharger une page web statique

### URL => PAGE HTML

La première étape est de se connecter à la page web et de télécharger le code source de la page.
On va donc à partir d'une url, télécharger une page html. 

Plusieurs modules pour faire des requêtes avec python existent: 
le plus simple étant le module [**requests**](http://docs.python-requests.org/en/master/)

Ce module télécharge la page demandée  à partir d'une url. Il donne aussi des informations contextuelles sur la requête: le code renvoyé par le serveur, ce qui peut être bien pratique en cas d'erreur.

Nous allons écrire une fonction download() qui prend une page web en argument et retourne True et la page **html** si tout c'est bien passé et False, et le code d'erreur HTPP si cela ne s'est pas bien passé.


In [ ]:
import requests
def download(url="http://www.marmiton.org/recettes/recette_veloute-chou-fleur-coco-sesame_346982.aspx"):
    response = requests.get(url)
    if response.status_code == 200:
        return (True, response.text)
    else:
        return (False, response.status_code)

In [ ]:
#on instancie la fonction avce une url
url = "http://www.marmiton.org/recettes/recette_veloute-chou-fleur-coco-sesame_346982.aspx"
response, page_html = download(url)

print(response)

## Identifier les informations pertinentes
### Page HTML => DOM Inspector

Nous avons récupéré la page HTML désirée, il s'agit pour le moment de HTML soit un arbre DOM (du texte et des balises HTML). Pour extraire les informations qui nous intéressent: 
il nous faut détecter les informations qui nous intéresse : dans quelles balises et à quel niveau du document
On utilise le DOM Inspector en général (l'exorateur de code de votre navigateur): 
Ouvrez votre navigateur: tapez F12 et inspecter la page web lister les informations que vous souhaitez extraire.

## Parser la page html

### Page HTML => DOM 

On a récupéré la page web, il faut parcourir le code HTML avec un parser. On va utiliser **BeautifulSoup** un module qui donne des méthodes simples pour accéder aux éléments html qui nous intéressent et qui repose sur un parser standard **lxml**. Nous les avons déjà installé nous allons donc mettre le code source de la page HTML dans un parser. Nous allons créer une fonction parse qui prend la page html et parser le code pour pouvoir le manipuler ensuite grace aux méthodes du module BeautifulSoup et extraure les informations dont on a besoin.


In [ ]:
from BeautifulSoup4 import BeautifulSoup
def parse(page_html):
    '''on transforme une page html en une *soupe* avce des tags quon peut rechercher '''
    soup = BeautifulSoup(page_html, "lxml")
    return soup

In [ ]:
#on instancie la fonction en se servant des fonctions déjà écrites

url = "http://www.marmiton.org/recettes/recette_veloute-chou-fleur-coco-sesame_346982.aspx"
response, page_html = download(url)
print(response)
soup = parse(page_html)


## Extraire les informations du HTML

### DOM  Tags=> elements textuels

A cette étape, il est utile de connaitre le HTML. 
Une fois qu'on a identifié les balises qui contiennent les éléments textuels qu'on veut extraire, on va les chercher et les stocker à l'aide des méthodes de BeautifulSoup find(), findAll(), get().

Ici on crée une fonction qui permet d'extraire:
- le titre de la page
- la liste des ingrédients, 
- le temps de préparation 
- le temps de cuission 
- les instructins de la recette 
en fonction du type de tag et de la classe de la balise.

In [ ]:
def extract_recette(soup):
    '''cette fonction est spécifique à marmiton'''
    titre = soup.find("h1", {"class":"m_title fn"})
    print("titre", titre.text)
    #ici on transforme une liste en chaine de caractère séparé par une virgule
    ingredients =  ",".join([n.text() for n in soup.findAll("a", {"class":"mrm_al"})])
    print("ingredients", ingredients)
    preparation_t = soup.find("span", {"class":"preptime"}).text()
    print("temps de preparation", preparation_t)
    cuisson_t = soup.find("span", {"class":"cooktime"}).text()
    print("temps de cuission", cuisson_t)
    instructions = soup.find("div", {"class": "m_content_recette_todo"}).text()
    print("instructions", instructions)
    return (titre, ingredients, preparation_t, cuisson_t, instructions)


In [ ]:
#on instancie la fonction en se servant des fonctions déjà écrites

url = "http://www.marmiton.org/recettes/recette_veloute-chou-fleur-coco-sesame_346982.aspx"
response, page_html = download(url)
print(response)
soup = parse(page_html)
recette = extract_recette(soup)


## Enregistrer et stocker les informations

### elements textuels => fichier
Une fois ses informatiosn extraite on peut la stocker en écrivant les résultats dans un fichier ou une base de données. Ici pour l'exemple ou va stocker la recette dans un CSV séparé par des tabulations en créant une fonction store_results qui prend les resultats et le nom du fichier.


In [ ]:
def store_results(results, file_name):
    #on ouvre un file descriptor et on met l'option 'a' pour append : soit écrire à la suite
    with open(file, "a") as f:
        #transforme la ligne de resultats en chaine de caractère séparée par une tabulation
        line = "\t".join(results)
        #ecris dans un fichier la ligne et ajoute un retour à la ligne
        f.write(+"\n")
    return 
    
#on instancie la fonction en se servant des fonctions déjà écrites
url = "http://www.marmiton.org/recettes/recette_veloute-chou-fleur-coco-sesame_346982.aspx"
response, page_html = download(url)
if response is True:
    soup = parse(page_html)
    recette = extract_recette(soup)
    store_results("./recettes.csv")


### Exercice